In [1]:
import os
path="/content/drive/MyDrive/COMP4211/PROJECT_xjs"
os.chdir(path)
os.listdir(path)

['test.csv',
 'test_model.py',
 'sample_submission.csv',
 'train.csv',
 'train.py',
 'ner_model',
 '__pycache__',
 'train_bert_large32',
 'tmp_trainer',
 'wandb',
 'train_bert_large64',
 'network.py',
 'data_loader.py',
 'microsoft',
 'submission.csv',
 'unhandled_tokens.txt',
 'skipped_rows.txt',
 'COMP4211 PROJ.ipynb']

In [2]:
!pip install datasets hf_xet seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import  get_linear_schedule_with_warmup
from tqdm import tqdm
from data_loader import prepare_data_loaders
from network import get_model
from transformers import AutoTokenizer
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from torch.cuda.amp import autocast, GradScaler
from seqeval.metrics import f1_score as seq_f1_score


In [4]:
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [10]:
# MODEL_NAME = "microsoft/deberta-v3-large"
# MODEL_NAME = "FacebookAI/roberta-base"
MODEL_NAME = "google-bert/bert-base-cased"
BATCH_SIZE = 32
EPOCHS = 3
USE_MIXED_PRECISION = False

# Step 1: Load data loaders and tokenizer
# Set total batch size to 32, to be split across GPUs
train_loader, eval_loader, label2id, num_labels, tokenizer = prepare_data_loaders(
    data_path=".",
    model_name=MODEL_NAME,
    batch_size=BATCH_SIZE,
    # apply_mistag_correction=True
)
o_tag_id = label2id.get('O', None)

# Step 2: Load model
model = get_model(MODEL_NAME, num_labels)

# Step 3: Set up device and DataParallel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_gpus = torch.cuda.device_count() if torch.cuda.is_available() else 0
if num_gpus > 1:
    print(f"Using {num_gpus} GPUs with DataParallel. Total batch size: {BATCH_SIZE}, per-GPU batch size: {BATCH_SIZE // num_gpus}")
    model = nn.DataParallel(model)
else:
    print(f"Using single GPU or CPU. Total batch size: {BATCH_SIZE}")
model.to(device)


Skipped 2 rows with mismatched lengths. Details in 'skipped_rows.txt'
Split data: 31998 training samples, 8000 validation samples


Map:   0%|          | 0/31998 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using single GPU or CPU. Total batch size: 32


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [11]:
# Step 4: Set up optimizer and scheduler
no_decay = ["bias", "LayerNorm.weight"]
pretrain_params = [
    {"params": [p for n, p in model.named_parameters() if "classifier" not in n and not any(nd in n for nd in no_decay)], "lr": 2e-5},
    {"params": [p for n, p in model.named_parameters() if "classifier" not in n and any(nd in n for nd in no_decay)], "lr": 2e-5, "weight_decay": 0.0},
]
classifier_params = [
    {"params": [p for n, p in model.named_parameters() if "classifier" in n and not any(nd in n for nd in no_decay)], "lr": 2e-4},
    {"params": [p for n, p in model.named_parameters() if "classifier" in n and any(nd in n for nd in no_decay)], "lr": 2e-4, "weight_decay": 0.0},
]
optimizer = AdamW(pretrain_params + classifier_params, lr=2e-5, fused=True)

total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=EPOCHS*0.1, num_training_steps=total_steps)
scaler = torch.amp.GradScaler("cuda" if torch.cuda.is_available() else "cpu", enabled=USE_MIXED_PRECISION)  # For mixed precision training

# Step 5: Training loop
# Initialize variables for tracking the best model
best_val_f1 = 0
best_model_state = None
patience = 3
epochs_no_improve = 0

for epoch in range(EPOCHS):
    # Training
    model.train()
    total_train_loss = 0
    train_preds = []
    train_labels_list = []
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.amp.autocast("cuda" if torch.cuda.is_available() else "cpu", enabled=USE_MIXED_PRECISION):
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()
        flat_preds = preds.flatten()
        flat_labels = labels_np.flatten()
        mask = flat_labels != o_tag_id
        train_preds.extend(flat_preds[mask])
        train_labels_list.extend(flat_labels[mask])

        loss = outputs.loss
        loss = loss.mean()  # Reduce loss to scalar for multi-GPU
        total_train_loss += loss.item()  # Log scalar loss

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        optimizer.zero_grad()

    avg_train_loss = total_train_loss / len(train_loader)
    train_f1 = f1_score(train_labels_list, train_preds, average='macro')
    # train_f1 = seq_f1_score(train_labels_list, train_preds)
    train_acc = accuracy_score(train_labels_list, train_preds)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, F1: {train_f1:.4f}, Acc: {train_acc:.4f}")

    # Validation
    model.eval()
    total_eval_loss = 0
    val_preds = []
    val_labels_list = []
    with torch.no_grad():
        for batch in tqdm(eval_loader, desc=f"Validation Epoch {epoch+1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1).detach().cpu().numpy()
            labels_np = labels.detach().cpu().numpy()
            flat_preds = preds.flatten()
            flat_labels = labels_np.flatten()
            mask = flat_labels != o_tag_id
            val_preds.extend(flat_preds[mask])
            val_labels_list.extend(flat_labels[mask])

            loss = outputs.loss
            loss = loss.mean()  # Reduce loss to scalar for multi-GPU
            total_eval_loss += loss.item()  # Log scalar loss

    avg_val_loss = total_eval_loss / len(eval_loader)
    val_f1 = f1_score(val_labels_list, val_preds, average='macro')
    # val_f1 = seq_f1_score(val_labels_list, val_preds)

    val_acc = accuracy_score(val_labels_list, val_preds)
    print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}, F1: {val_f1:.4f}, Acc: {val_acc:.4f}")

    # If the current validation loss is lower than the previous best, save the model state
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        epochs_no_improve = 0
        # Save the state of the underlying model (not the DataParallel wrapper)
        best_model_state = model.module.state_dict().copy() if isinstance(model, nn.DataParallel) else model.state_dict().copy()
        print(f"New best model found at epoch {epoch+1} with f1 score: {val_f1:.4f}")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

# After training, load the best model state
if best_model_state is not None:
    if isinstance(model, nn.DataParallel):
        model.module.load_state_dict(best_model_state)
    else:
        model.load_state_dict(best_model_state)
    print(f"Loaded best model with f1 score: {best_val_f1}")

# Step 6: Save the best model and tokenizerz
# Save the underlying model (not the DataParallel wrapper)
model_to_save = model.module if isinstance(model, nn.DataParallel) else model
model_to_save.save_pretrained(MODEL_NAME + "_best_model")
tokenizer.save_pretrained(MODEL_NAME + "_best_model")

Training Epoch 1: 100%|██████████| 1000/1000 [04:41<00:00,  3.55it/s]


Epoch 1, Training Loss: 0.1543, F1: 0.1153, Acc: 0.0110


Validation Epoch 1: 100%|██████████| 250/250 [00:29<00:00,  8.47it/s]


Epoch 1, Validation Loss: 0.0975, F1: 0.1467, Acc: 0.0118
New best model found at epoch 1 with f1 score: 0.1467


Training Epoch 2: 100%|██████████| 1000/1000 [04:41<00:00,  3.55it/s]


...

Training Epoch 3: 100%|██████████| 1000/1000 [04:41<00:00,  3.55it/s]


Epoch 3, Training Loss: 0.0684, F1: 0.2135, Acc: 0.0124


Validation Epoch 3: 100%|██████████| 250/250 [00:29<00:00,  8.49it/s]


Epoch 3, Validation Loss: 0.0914, F1: 0.1609, Acc: 0.0121
New best model found at epoch 3 with f1 score: 0.1609
Loaded best model with f1 score: 0.16090943292104154


('google-bert/bert-base-cased_best_model/tokenizer_config.json',
 'google-bert/bert-base-cased_best_model/special_tokens_map.json',
 'google-bert/bert-base-cased_best_model/vocab.txt',
 'google-bert/bert-base-cased_best_model/added_tokens.json',
 'google-bert/bert-base-cased_best_model/tokenizer.json')

In [12]:
# Manual save if interrupted
if best_model_state is not None:
    if isinstance(model, nn.DataParallel):
        model.module.load_state_dict(best_model_state)
    else:
        model.load_state_dict(best_model_state)
    print(f"Loaded best model with f1 score: {best_val_f1}")
model_to_save = model.module if isinstance(model, nn.DataParallel) else model
model_to_save.save_pretrained(MODEL_NAME + "_best_model")
tokenizer.save_pretrained(MODEL_NAME + "_best_model")

Loaded best model with f1 score: 0.16090943292104154


('google-bert/bert-base-cased_best_model/tokenizer_config.json',
 'google-bert/bert-base-cased_best_model/special_tokens_map.json',
 'google-bert/bert-base-cased_best_model/vocab.txt',
 'google-bert/bert-base-cased_best_model/added_tokens.json',
 'google-bert/bert-base-cased_best_model/tokenizer.json')

In [12]:
torch.cuda.empty_cache()

In [13]:
import pandas as pd
from ast import literal_eval
from datasets import Dataset
from transformers import AutoModelForTokenClassification, Trainer
import numpy as np
from data_loader import create_label_mappings, normalize_token
from data_loader import load_and_preprocess_data

In [14]:
def load_test_data(file_path):
    """Load and preprocess test data, normalizing non-standard characters."""
    try:
        df = pd.read_csv(file_path)
        df['Sentence'] = df['Sentence'].apply(literal_eval)
        df['Sentence'] = df['Sentence'].apply(lambda x: [normalize_token(w.strip()) for w in x if w.strip()])
        return df
    except Exception as e:
        print(f"Failed to load test data from {file_path}: {e}")
        raise

def predict_and_save(model_path, test_ds, train_df, output_file="submission.csv"):
    """Predict NER tags and save submission."""
    try:
        # Load model and tokenizer
        model = AutoModelForTokenClassification.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
    except Exception as e:
        print(f"Failed to load model or tokenizer from {model_path}: {e}")
        raise

    # Load label mappings from training data
    try:
        label_list, label2id, id2label, _ = create_label_mappings(train_df)
    except Exception as e:
        print(f"Failed to create label mappings: {e}")
        id2label = model.config.id2label
        label2id = model.config.label2id
        if not id2label:
            print("Warning: id2label not found in model config. Using default labels.")
            id2label = {i: f"LABEL_{i}" for i in range(model.config.num_labels)}
        label_list = list(label2id.keys())

    def tokenize(examples):
        """Tokenize sentences for prediction."""
        tokenized = tokenizer(
            examples["Sentence"],
            truncation=True,
            is_split_into_words=True,
            padding="max_length",
            max_length=256,
            return_special_tokens_mask=True
        )
        return tokenized

    try:
        # Tokenize test dataset
        tokenized_test = test_ds.map(tokenize, batched=True, remove_columns=["Sentence", "id"])
    except Exception as e:
        print(f"Failed to tokenize test dataset: {e}")
        raise

    # Initialize Trainer
    trainer = Trainer(model=model, tokenizer=tokenizer)

    try:
        # Predict
        predictions = trainer.predict(tokenized_test)
        preds = np.argmax(predictions.predictions, axis=2)  # Convert logits to label IDs
    except Exception as e:
        print(f"Prediction failed: {e}")
        raise

    # Process predictions
    final_preds = []
    test_df = test_ds.to_pandas()

    for i in range(len(test_ds)):
        sentence = test_ds[i]["Sentence"]
        sentence_length = len(sentence)
        pred_ids = preds[i]

        # Re-tokenize to get word_ids
        encoding = tokenizer(
            sentence,
            truncation=True,
            is_split_into_words=True,
            padding=False,
            max_length=256,
            return_special_tokens_mask=True
        )
        word_ids = encoding.word_ids()

        # Align predictions with words
        word_to_tag = {}
        for idx, word_idx in enumerate(word_ids):
            if word_idx is None:
                continue
            if word_idx not in word_to_tag:
                pred_tag = id2label[pred_ids[idx]] if pred_ids[idx] != -100 else 'O'
                word_to_tag[word_idx] = pred_tag

        # Assign tags to each word
        word_labels = []
        for word_idx in range(sentence_length):
            normalized = normalize_token(sentence[word_idx])
            if normalized in ["'s", ' ', '-', '"', ''] or not normalized.strip():
                word_labels.append("O")
            else:
                word_labels.append(word_to_tag.get(word_idx, "O"))

        # Format as JSON-like string
        word_labels_str = str(word_labels)
        final_preds.append({'id': test_ds[i]['id'], 'NER Tag': word_labels_str})

    # Save submission
    try:
        submission_df = pd.DataFrame(final_preds)
        # Validate lengths
        for idx, row in submission_df.iterrows():
            sentence = test_ds[idx]["Sentence"]
            predicted_tags = literal_eval(row["NER Tag"])
            if len(predicted_tags) != len(sentence):
                print(f"Warning: ID {row['id']} has mismatched lengths: "
                      f"Predicted {len(predicted_tags)}, Sentence {len(sentence)}")
        submission_df.to_csv(output_file, index=False)
        print(f"Submission saved to {output_file}")
    except Exception as e:
        print(f"Failed to save submission to {output_file}: {e}")
        raise


# Test the best model and save predictions

In [15]:
# Configuration
model_path = MODEL_NAME + "_best_model"
test_file = "test.csv"
train_file = "train.csv"
output_file = "submission.csv"

# Load test and train data
test_df = load_test_data(test_file)
test_ds = Dataset.from_pandas(test_df)
print(f"Test dataset size: {len(test_ds)}")

train_df = load_and_preprocess_data(train_file)
print(f"Train dataset size: {len(train_df)}")

# Predict and save
predict_and_save(model_path, test_ds, train_df, output_file)

Test dataset size: 5000
Skipped 2 rows with mismatched lengths. Details in 'skipped_rows.txt'
Train dataset size: 39998


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

<ipython-input-14-fccda80a700e>:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


Submission saved to submission.csv


In [ ]:
from google.colab import runtime
runtime.unassign()